## Домашнее задание
1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности
### Ссылки
http://hyperopt.github.io/hyperopt/ <br>
https://arxiv.org/pdf/1907.03947.pdf <br>
https://arxiv.org/pdf/1802.02301.pdf <br>
https://arxiv.org/list/stat.ML/recent <br>
https://scikit-learn.org/stable/modules/grid_search.html <br>
https://scikit-learn.org/stable/modules/compose.html <br>

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)

In [3]:
df = pd.read_csv("churn_data.csv")
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [4]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

- Категориальные признаки закодируем с помощью OneHotEncoding
- Вещественные масштабируем с помощью StandardScaler

In [5]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [6]:
# Зададим списки признаков
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [7]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [8]:
# Объединим все это в единый пайплайн
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [9]:
models_results = pd.DataFrame()
models_results['model'] = []
models_results['Precision'] = []
models_results['Recall'] = []
models_results['F-Score'] = []
models_results['ROC_AUC'] = []

Бустинг

In [10]:
pipeline_1 = Pipeline([
    ('features',feats),
    ('classifier', xgb.XGBClassifier(random_state = 42, use_label_encoder=False, objective='binary:logistic', eval_metric='logloss')),
])

In [11]:
#обучим наш пайплайн
pipeline_1.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [12]:
#наши прогнозы для тестовой выборки
preds_1 = pipeline_1.predict_proba(X_test)[:, 1]
preds_1[:10]

array([0.47611395, 0.23087966, 0.07372608, 0.03141731, 0.02561874,
       0.9679959 , 0.06951059, 0.2885457 , 0.11367497, 0.4902508 ],
      dtype=float32)

In [13]:
precision_1, recall_1, thresholds_1 = precision_recall_curve(y_test, preds_1)

fscore_1 = (2 * precision_1 * recall_1) / (precision_1 + recall_1)
# locate the index of the largest f score
ix_1 = np.argmax(fscore_1)
roc_auc_1 = roc_auc_score(y_test, preds_1)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, ROC_AUC=%.3f' % (thresholds_1[ix_1], 
                                                                        fscore_1[ix_1],
                                                                        precision_1[ix_1],
                                                                        recall_1[ix_1],
                                                                        roc_auc_1))

Best Threshold=0.373318, F-Score=0.626, Precision=0.619, Recall=0.633, ROC_AUC=0.861


In [14]:
models_results = models_results.append({'model': 'XGBClassifier', 
                'Precision': precision_1[ix_1],
                'Recall': recall_1[ix_1],
                'F-Score': fscore_1[ix_1],
                'ROC_AUC': roc_auc_1},
                ignore_index=True)
models_results

,model,Precision,Recall,F-Score,ROC_AUC
0,XGBClassifier,0.619231,0.632613,0.62585,0.860921


Логистическая регрессия

In [15]:
pipeline_2 = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

In [16]:
#обучим наш пайплайн
pipeline_2.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [17]:
#наши прогнозы для тестовой выборки
preds_2 = pipeline_2.predict_proba(X_test)[:, 1]
preds_2[:10]

array([0.22805865, 0.3350657 , 0.15347886, 0.12466446, 0.15507743,
       0.6431308 , 0.06214346, 0.077948  , 0.36717132, 0.76751542])

In [18]:
precision_2, recall_2, thresholds_2 = precision_recall_curve(y_test, preds_2)

fscore_2 = (2 * precision_2 * recall_2) / (precision_2 + recall_2)
# locate the index of the largest f score
ix_2 = np.argmax(fscore_2)
roc_auc_2 = roc_auc_score(y_test, preds_2)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, ROC_AUC=%.3f' % (thresholds_2[ix_2], 
                                                                        fscore_2[ix_2],
                                                                        precision_2[ix_2],
                                                                        recall_2[ix_2],
                                                                        roc_auc_2))

Best Threshold=0.289522, F-Score=0.510, Precision=0.462, Recall=0.568, ROC_AUC=0.772


In [19]:
models_results = models_results.append({'model': 'LogisticRegression', 
                'Precision': precision_2[ix_2],
                'Recall': recall_2[ix_2],
                'F-Score': fscore_2[ix_2],
                'ROC_AUC': roc_auc_2},
                ignore_index=True)
models_results

,model,Precision,Recall,F-Score,ROC_AUC
0,XGBClassifier,0.619231,0.632613,0.62585,0.860921
1,LogisticRegression,0.462400,0.567780,0.50970,0.772077


Случайный лес

In [20]:
pipeline_3 = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

In [21]:
#обучим наш пайплайн
pipeline_3.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [22]:
#наши прогнозы для тестовой выборки
preds_3 = pipeline_3.predict_proba(X_test)[:, 1]
preds_3[:10]

array([0.37, 0.26, 0.17, 0.02, 0.02, 0.67, 0.04, 0.12, 0.15, 0.75])

In [23]:
precision_3, recall_3, thresholds_3 = precision_recall_curve(y_test, preds_3)

fscore_3 = (2 * precision_3 * recall_3) / (precision_3 + recall_3)
# locate the index of the largest f score
ix_3 = np.argmax(fscore_3)
roc_auc_3 = roc_auc_score(y_test, preds_3)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, ROC_AUC=%.3f' % (thresholds_3[ix_3], 
                                                                        fscore_3[ix_3],
                                                                        precision_3[ix_3],
                                                                        recall_3[ix_3],
                                                                        roc_auc_3))

Best Threshold=0.380000, F-Score=0.641, Precision=0.654, Recall=0.629, ROC_AUC=0.864


In [24]:
models_results = models_results.append({'model': 'RandomForestClassifier', 
                'Precision': precision_3[ix_3],
                'Recall': recall_3[ix_3],
                'F-Score': fscore_3[ix_3],
                'ROC_AUC': roc_auc_3},
                ignore_index=True)
models_results

,model,Precision,Recall,F-Score,ROC_AUC
0,XGBClassifier,0.619231,0.632613,0.625850,0.860921
1,LogisticRegression,0.462400,0.567780,0.509700,0.772077
2,RandomForestClassifier,0.654397,0.628684,0.641283,0.863699


2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)

Наверное, f-мера подходит больше всех: нам нужно найти баланс между точностью предсказания(т.е. precision) и охватом(т.е. recall), так как нам неинтересно удерживать тех, кто и так не уйдет, но и упускать большую долю тех, кто точно уйдет тоже не хочется. 

In [25]:
models_results.sort_values(['F-Score'], ascending=False)

,model,Precision,Recall,F-Score,ROC_AUC
2,RandomForestClassifier,0.654397,0.628684,0.641283,0.863699
0,XGBClassifier,0.619231,0.632613,0.625850,0.860921
1,LogisticRegression,0.462400,0.567780,0.509700,0.772077


Лучшим оказался случайный лес, но его мы разбирали на вебинаре, а из домашних моделей лучшая - бустинг.

3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.

In [26]:
cnf_matrix = confusion_matrix(y_test, preds_1>thresholds_1[ix_1])
cnf_matrix

array([[1793,  198],
       [ 188,  321]])

In [27]:
print(f'TP = {cnf_matrix[0][0]} - думаем уйдут -> уйдут')
print(f'FP = {cnf_matrix[0][1]} - думаем уйдут -> не уйдут')
print(f'FN = {cnf_matrix[1][0]} - думаем не уйдут -> уйдут')
print(f'TN = {cnf_matrix[1][1]} - думаем не уйдут -> не уйдут')

TP = 1793 - думаем уйдут -> уйдут
FP = 198 - думаем уйдут -> не уйдут
FN = 188 - думаем не уйдут -> уйдут
TN = 321 - думаем не уйдут -> не уйдут


С удержанием:<br>
Прибыль принесут только TP: потратим на удержание - получим доход. TP: $-1793*1 + 1793*2$<br>
FP: потратим на удержание, доход получим. FP: $-198*1 + 198*2$<br>
FN: не потратим на удержание, но потеряем доход с них. FN: $-188*2$<br>
TN: не потратим на удержание, не потеряем доход: TN: $321*2$<br><br>
Без удержания:<br>
TP и FN уйдут и мы потеряем прибыль от них, останется только прибыль от FP и TN: $-2*1793 - 2*188 + 2*198 + 2*321$

In [28]:
res_with_model = 1793 + 198 - 2*188 + 321*2
res_without_model = -2*1793 - 2*188 + 2*198 + 2*321
print(f'Прибыль с удержанием: {res_with_model}')
print(f'Прибыль без удержания: {res_without_model}')

Прибыль с удержанием: 2257
Прибыль без удержания: -2924


Получилось, что наша модель потенциально экономически целесообразна.

4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3

In [29]:
params={'classifier__learning_rate':[0.01, 0.3],
        'classifier__gamma':[0, 0.5],
        'classifier__max_depth':[None, 3, 10],
        }

In [30]:
%%time
grid = GridSearchCV(pipeline_1,
                    param_grid=params,
                    cv=6,
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

CPU times: user 4min 49s, sys: 3.02 s, total: 4min 52s
Wall time: 1min 22s


{'classifier__gamma': 0.5,
 'classifier__learning_rate': 0.3,
 'classifier__max_depth': 3}

In [31]:
pipeline_4 = Pipeline([
    ('features',feats),
    ('classifier', xgb.XGBClassifier(random_state = 42, 
                                     use_label_encoder=False, 
                                     objective='binary:logistic', 
                                     eval_metric='logloss',
                                     gamma=0.5, 
                                     learning_rate=0.3, 
                                     max_depth=3)),
])

In [32]:
#обучим наш пайплайн
pipeline_4.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Geography',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))])),
                                                ('Gender',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Gender'))])),
                                                ('Tenure',
                                                 Pipeline(steps=[('selector',
           

In [33]:
#наши прогнозы для тестовой выборки
preds_4 = pipeline_4.predict_proba(X_test)[:, 1]
preds_4[:10]

array([0.23872866, 0.18331882, 0.19888023, 0.07336348, 0.01720506,
       0.8186907 , 0.03651758, 0.07182581, 0.21028003, 0.7489    ],
      dtype=float32)

In [34]:
precision_4, recall_4, thresholds_4 = precision_recall_curve(y_test, preds_4)

fscore_4 = (2 * precision_4 * recall_4) / (precision_4 + recall_4)
# locate the index of the largest f score
ix_4 = np.argmax(fscore_4)
roc_auc_4 = roc_auc_score(y_test, preds_4)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, ROC_AUC=%.3f' % (thresholds_4[ix_4], 
                                                                        fscore_4[ix_4],
                                                                        precision_4[ix_4],
                                                                        recall_4[ix_4],
                                                                        roc_auc_4))

Best Threshold=0.332742, F-Score=0.646, Precision=0.632, Recall=0.660, ROC_AUC=0.874


In [35]:
models_results = models_results.append({'model': 'XGBClassifier_best', 
                'Precision': precision_4[ix_4],
                'Recall': recall_4[ix_4],
                'F-Score': fscore_4[ix_4],
                'ROC_AUC': roc_auc_4},
                ignore_index=True)
models_results

,model,Precision,Recall,F-Score,ROC_AUC
0,XGBClassifier,0.619231,0.632613,0.625850,0.860921
1,LogisticRegression,0.462400,0.567780,0.509700,0.772077
2,RandomForestClassifier,0.654397,0.628684,0.641283,0.863699
3,XGBClassifier_best,0.631579,0.660118,0.645533,0.874029


5. (опционально) Еще раз провести оценку экономической эффективности

In [36]:
cnf_matrix = confusion_matrix(y_test, preds_4>thresholds_4[ix_4])
cnf_matrix

array([[1795,  196],
       [ 174,  335]])

In [37]:
print(f'TP = {cnf_matrix[0][0]} - думаем уйдут -> уйдут')
print(f'FP = {cnf_matrix[0][1]} - думаем уйдут -> не уйдут')
print(f'FN = {cnf_matrix[1][0]} - думаем не уйдут -> уйдут')
print(f'TN = {cnf_matrix[1][1]} - думаем не уйдут -> не уйдут')

TP = 1795 - думаем уйдут -> уйдут
FP = 196 - думаем уйдут -> не уйдут
FN = 174 - думаем не уйдут -> уйдут
TN = 335 - думаем не уйдут -> не уйдут


С удержанием:<br>
Прибыль принесут только TP: потратим на удержание - получим доход. TP: $-1795*1 + 1795*2$<br>
FP: потратим на удержание, доход получим. FP: $-196*1 + 196*2$<br>
FN: не потратим на удержание, но потеряем доход с них. FN: $-174*2$<br>
TN: не потратим на удержание, не потеряем доход: TN: $335*2$<br><br>
Без удержания:<br>
TP и FN уйдут и мы потеряем прибыль от них, останется только прибыль от FP и TN: $-2*1795 - 2*174 + 2*196 + 2*335$

In [38]:
res_with_model = 1795 + 196 - 2*174 + 335*2
res_without_model = -2*1795 - 2*174 + 2*196 + 2*335
print(f'Прибыль с удержанием: {res_with_model}')
print(f'Прибыль без удержания: {res_without_model}')

Прибыль с удержанием: 2313
Прибыль без удержания: -2876


Было:<br>
Прибыль с удержанием: 2257<br>
Прибыль без удержания: -2924

Выгода от модели немного увеличилась.